# railファイル確認用

In [19]:
import boto3
import os
import importlib
import shelve
import copy
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
import datetime
# import src.utilsST_01 as utlst
import src.helpers as helpers
import src.visualize as vis
from src.config import appProperties
from src.trolley import trolley
from src.similar_pixel import pixel
from src.kalman import kalman
from src.logger import my_logger
from pprint import pprint
importlib.reload(helpers)
importlib.reload(vis)

config = appProperties('config.yml')

In [2]:
images_path = helpers.list_imagespath_nonCache(config.image_dir)
# dir_area = "Chuo_01_Tokyo-St_up_20230201_knight"
# dir_area = "Chuo_02_Tokyo-Kanda_up_20230201_knight"
# dir_area = "Kawagoe_02_Nishioomiya-Sashiougi_down_20220626_knight"
# dir_area = "Chuo_24_Higashinakano-Nakano_down_20230201_day"
# dir_area = "Takasaki_01_Omiya-Miyahara_up_20230217_day"
# dir_area = "Yamate_49_Takadanobaba-st_soto_20230129_knight"
dir_area = "Yamate_40_Shibuya-st_soto_20230129_knight"
camera_num = "HD11"
target_dir = config.image_dir + "/" + dir_area + "/" + camera_num
outpath = config.output_dir + "/" + dir_area + "/" + camera_num

base_images = helpers.list_images(target_dir)

rail_fpath = outpath + "/rail.shelve"

# with shelve.open(rail_fpath) as rail:
#     rail_dict = copy.deepcopy(rail)

# rail = shelve.open(rail_fpath)

In [ ]:
len(base_images)

# shelveファイルの中身を1階層ずつチェックする

In [ ]:
with shelve.open(rail_fpath) as rail:
    rail_dict = copy.deepcopy(rail)

keys = rail_dict.keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
keys = rail_dict[camera_num].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
image_path = base_images[0]
image_path

In [ ]:
keys = rail_dict[camera_num][image_path].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

In [ ]:
len(trolley_dict)

In [ ]:
# for trolley_id in config.trolley_ids[:len(trolley_dict)]:
for trolley_id in trolley_dict.keys():
    print(trolley_id)
    for key in trolley_dict[trolley_id].keys():
        print(key)

In [ ]:
trolley_dict['imgs/Yamate_40_Shibuya-st_soto_20230129_knight/HD12/2022_0599_HD12_01_00009117.jpg']

In [ ]:
trolley_dict[]["estimated_upper_edge"]

In [ ]:
keys = rail_dict[camera_num][image_path]["trolley1"].keys()
for i, key in enumerate(keys):
    # print(f'{i}: {key}')
    try:
        key_len = len(rail_dict[camera_num][image_path]["trolley1"][key])
    except:
        key_len = 0
    print(f'{key} > length:{key_len}')

In [ ]:
image_path = base_images[0]
rail["HD11"][image_path]["trolley1"]["estimated_upper_edge"]

In [ ]:
'''
case1: xのみnp.array   -> 2023-05-24 00:44:31.574638 Process end :0:00:45.240699
case2: x,yをnp.array   -> 2023-05-24 02:24:05.555617 Process end :0:00:57.180313
case3: rail_dict化     -> 2023-05-24 04:58:30.819891 Process end :0:00:07.624567
case4: shelve.open追加 -> 2023-05-24 05:17:12.809013 Process end :0:00:03.504746
'''

dt01 = datetime.datetime.now()

img_num = 50

with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

for idx, image_path in enumerate(base_images):
    print(f"{idx}> {image_path}")
    x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
    for trolley_id in config.trolley_ids:
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])
    if idx >= img_num-1:
        print("break")
        break

plt.show()

dt02 = datetime.datetime.now()
prc_time = dt02 - dt01
print(str(datetime.datetime.now()) + f' Process end :{prc_time}')

In [ ]:
for key in trolley_dict[base_images[0]]['trolley1'].keys():
    print(key)

In [ ]:
print("最後の検出結果")
for trolley_id in config.trolley_ids:
    print(f"trolley_id:{trolley_id}")
    print(f'estimated_upper_edge:{trolley_dict[image_path][trolley_id]["estimated_upper_edge"][999]}')
    print(f'estimated_lower_edge:{trolley_dict[image_path][trolley_id]["estimated_lower_edge"][999]}')
    print(f'brightness_std:{trolley_dict[image_path][trolley_id]["brightness_std"][999]}')
    print(f'brightness_center:{trolley_dict[image_path][trolley_id]["brightness_center"][999]}')

In [ ]:
%%time
with shelve.open(rail_fpath) as rail:
    if len(rail) < 2 or not any(len(rail[camera_num].get(image_path, {})) > 0 for image_path in base_images):
        print('rail initilize')
        print(f'dir_area: {rail["name"]}')
        # railを初期化
        rail[camera_num] = {image_path: {trolley_id: {} for trolley_id in trolley_ids} for image_path in base_images}

In [ ]:
%%time
with shelve.open(rail_fpath) as rail:
    if len(rail) < 2:    # 初めてrailが生成された場合は"name"だけなのでlen(rail)は1
            rail_check = False
    else:    # 一度でも解析されるとtrolley_idが追加されるため1以上
        rail_check = any(len(rail[camera_num][image_path]) > 0 for image_path in base_images)
    if not rail_check:
        print('rail initilize')
        print(f'dir_area: {rail["name"]}')
        # railを初期化
        # base_imagesと同じ長さの空のdictionaryを作成してrailを初期化
        blankdict_size = [{}] * len(base_images)
        rail[camera_num] = dict(zip(base_images, blankdict_size))
        # trolley_idsと同じ長さの空のdictionaryを作成してrailを初期化
        blankdict_size = [{}] * len(trolley_ids)
        for image_path in base_images:
            rail[camera_num][image_path] = dict(zip(trolley_ids, blankdict_size))

In [ ]:
%%time
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.copy(rail[camera_num])

rail_check = any(len(trolley_dict[image_path]) > 0 for image_path in base_images)
print(rail_check)

In [ ]:
%%time
with shelve.open(rail_fpath) as rail:
    rail_check = any(len(rail[camera_num][image_path]) > 0 for image_path in base_images)
print(rail_check)

# 結果画像を表示する

In [ ]:
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
for trolley_id in config.trolley_ids:
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])


In [ ]:
# 画像とデータの読み込み
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

# 画像をnumpy配列に変換
img_array = np.array(img)

# ランダムに1000画素を選択し、その平均輝度を背景の輝度とする
random_pixels = img_array[np.random.randint(0, img_array.shape[0], 1000), np.random.randint(0, img_array.shape[1], 1000)]
background_brightness = random_pixels.mean()

# データを描画
x_values = trolley_dict[image_path][config.trolley_ids[0]]["ix"]
for trolley_id in config.trolley_ids:
    upper_edge = trolley_dict[image_path][trolley_id]["estimated_upper_edge"]
    lower_edge = trolley_dict[image_path][trolley_id]["estimated_lower_edge"]
    for x, y1, y2 in zip(x_values, upper_edge, lower_edge):
        # estimated_upper_edgeとestimated_lower_edgeが0でない場合のみ色を変更
        if y1 != 0:
            color_upper = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y1, x] = color_upper
        if y2 != 0:
            color_lower = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y2, x] = color_lower

# 変更後の画像を表示
display_img = Image.fromarray(img_array)
display_img

# 結果をCSV出力する

In [ ]:
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

In [ ]:
base_images = list(trolley_dict.keys())
print(f"Images count: {len(base_images)}")
img_path = base_images[int(input("set index number: "))]
print(f"img_path: {img_path}")

In [ ]:
config.trolley_ids

In [ ]:
print(f"img_path: {trolley_dict[img_path]}")
for key in trolley_dict[img_path].keys():
    print(key)

In [ ]:
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    print(key)

In [ ]:
trolley_id = config.trolley_ids[0]
key = "estimated_upper_edge"
print(trolley_id)
print(type(trolley_dict[img_path][trolley_id][key]))
print(len(trolley_dict[img_path][trolley_id][key]))
print(trolley_dict[img_path][trolley_id][key])

In [ ]:
# 単体テスト用
# shelveをCSVファイルとして書き出す

# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv_fpath:{csv_fpath}")

# 1画像あたりの間引く数を指定
thin_out = 50

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

In [ ]:
img_path = 'imgs/Takasaki_01_Omiya-Miyahara_up_20230217_day/HD11/2022_0599_HD11_01_00012454.jpg'
trolley_dict[img_path].keys()

In [ ]:
trolley_id = config.trolley_ids[0]
column_name = [trolley_id + "_" + key for key in list(trolley_dict[img_path][trolley_id].keys())]
column_name

In [ ]:
column_name = ["ix"]
for trolley_id in config.trolley_ids:
    trolley_column = [trolley_id + "_" + key for key in config.result_keys]
    column_name = column_name + trolley_column
print(column_name)

In [ ]:
column_name = [
    f"{trolley_id}_{key}"
    for trolley_id in config.trolley_ids
    for key in config.result_keys
]
print(column_name)

In [ ]:
df = pd.DataFrame(np.nan, index=np.arange(config.max_len), columns=column_name)
df

In [ ]:
trolley_id = config.trolley_ids[0]
helpers.trim_trolley_dict(config, trolley_dict, img_path)
df = pd.DataFrame(trolley_dict[img_path][trolley_id]).copy()

In [ ]:
df

In [ ]:
for i, key in enumerate(config.result_keys):
    print(f'{i}> key:{key} ->{bool(key in df.columns)}')

In [ ]:
len(config.result_keys)

In [ ]:
len(df.columns)

In [ ]:
trolley_id = config.trolley_ids[0]
df1 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df1.columns)))
for column in diff_list:
    df1[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df1 = df1.reindex(columns=config.result_keys, copy=False)
df1.columns

In [ ]:
trolley_id = config.trolley_ids[0]
df2 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df2.columns)))
for column in diff_list:
    df2[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df2 = df2.reindex(columns=config.result_keys, copy=False)
df2.columns

In [ ]:
trolley_id = config.trolley_ids[0]
df3 = pd.DataFrame(trolley_dict[img_path][trolley_id])
diff_list = list(set(config.result_keys) - set(list(df3.columns)))
for column in diff_list:
    df3[column] = pd.Series([np.nan]*config.max_len, index=df.index)
df3 = df3.reindex(columns=config.result_keys, copy=False)
df3.columns

In [ ]:
df_trolley = pd.DataFrame()
df_trolley = pd.concat([df_trolley, df1], axis=1)
df_trolley = pd.concat([df_trolley, df2], axis=1)
df_trolley = pd.concat([df_trolley, df3], axis=1)
df_trolley.columns

In [ ]:
df_trolley.columns = column_name
df_trolley.columns

In [ ]:
len(df_trolley.columns)

In [ ]:
len(df_trolley.columns)

In [ ]:
len(column_name)

In [ ]:
dfs = pd.DataFrame()
for img_idx, img_path in enumerate(base_images):
    # print(f"{img_idx}> img_path: {img_path}")
    if len(trolley_dict[img_path]):
        # img_pathの結果が空でないときに実行する
        df_trolley = helpers.read_trolley_dict(config, trolley_dict, img_idx, img_path, thin_out).copy()
        # ixの値が連番になるように修正
        df_trolley['ix'] = df_trolley['ix'] + 1000 * img_idx
    elif img_idx:
        # img_pathの結果が空のとき & img_idxが0以外のとき
        # print("trolley_dict[img_path] is empty")
        df_trolley = helpers.trolley_dict_fillna(img_idx, img_path, dfs.columns).copy()
    elif img_idx:
        # 1枚目の画像でエラーになる
        print("解析結果がないため、CSVファイルを生成できませんでした。")
        break
    dfs = pd.concat([dfs, df_trolley], ignore_index=True)

# estimated_widthの標準偏差を計算して追記する
dfs = helpers.width_std_calc(config, dfs, thin_out)
# データを間引く
dfs = dfs[::thin_out]
print("convert shelve to dataframe")
dfs

In [ ]:
dfs.columns

In [ ]:
# CSVファイルに変換する
dfs.to_csv(csv_fpath, encoding='cp932')

# CSVファイルからグラフを表示する

In [ ]:
csv_fpath = 

In [ ]:
from bokeh.plotting import figure, gridplot, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import d3
output_notebook()


def plot_fig_bokeh(csv_fpath, graph_height, graph_width):

    # CSVファイルをデータフレームとして読み込む
    df_csv = pd.read_csv(csv_fpath, encoding='cp932')
    
    # CSVから作成したデータフレームをbokeh形式で読み込む
    source = ColumnDataSource(data=df_csv)
    
    # グラフの色情報を設定する
    # グラフの数に合わせて12色だけ取得する
    colors = d3["Category20"][12]

    # ツールチップを設定
    TOOLTIPS_EDGE=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('upper_edge1', '@trolley1_estimated_upper_edge'),
        ('lower_edge1', '@trolley1_estimated_lower_edge'),
        ('upper_edge2', '@trolley2_estimated_upper_edge'),
        ('lower_edge2', '@trolley2_estimated_lower_edge'),
        ('upper_edge3', '@trolley3_estimated_upper_edge'),
        ('lower_edge3', '@trolley3_estimated_lower_edge'),
    ]
    TOOLTIPS_WIDTH=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('estimated_width1', '@trolley1_estimated_width'),
        ('estimated_width2', '@trolley2_estimated_width'),
        ('estimated_width3', '@trolley3_estimated_width'),
    ]
    TOOLTIPS_BRIGHTNESS=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('brightness_center1', '@trolley1_brightness_center'),
        ('brightness_center2', '@trolley2_brightness_center'),
        ('brightness_center3', '@trolley3_brightness_center'),
    ]

    # グラフを作成
    p_edge = figure(
        title="Upper and Lower Edge",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_EDGE,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_width = figure(
        title="Width",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_WIDTH,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_center = figure(
        title="Brightness Center",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_BRIGHTNESS,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )

    # グラフを表示する領域を作成
    grid = gridplot(
        [[p_edge], [p_width], [p_center]],
        toolbar_location="above"
    )

    # データを追加
    x_values = "ix"
    upper_edge1 = "trolley1_estimated_upper_edge"
    lower_edge1 = "trolley1_estimated_lower_edge"
    upper_edge2 = "trolley2_estimated_upper_edge"
    lower_edge2 = "trolley2_estimated_lower_edge"
    upper_edge3 = "trolley3_estimated_upper_edge"
    lower_edge3 = "trolley3_estimated_lower_edge"
    estimated_width1 = "trolley1_estimated_width"
    estimated_width2 = "trolley2_estimated_width"
    estimated_width3 = "trolley3_estimated_width"
    brightness_center1 = "trolley1_brightness_center"
    brightness_center2 = "trolley2_brightness_center"
    brightness_center3 = "trolley3_brightness_center"

    # グラフにデータを追加
    # 1つ目のグラフ（Upper and Lower Edge）
    p_edge.line(x_values, upper_edge1, line_color=colors[0], source=source)
    p_edge.line(x_values, upper_edge2, line_color=colors[1], source=source)
    p_edge.line(x_values, upper_edge3, line_color=colors[2], source=source)
    p_edge.line(x_values, lower_edge1, line_color=colors[3], source=source)
    p_edge.line(x_values, lower_edge2, line_color=colors[4], source=source)
    p_edge.line(x_values, lower_edge3, line_color=colors[5], source=source)

    # 2つ目のグラフ（Brightness Std）
    p_width.line(x_values, estimated_width1, line_color=colors[6], source=source)
    p_width.line(x_values, estimated_width2, line_color=colors[7], source=source)
    p_width.line(x_values, estimated_width3, line_color=colors[8], source=source)

    # 3つ目のグラフ（Brightness Center）
    p_center.line(x_values, brightness_center1, line_color=colors[9], source=source)
    p_center.line(x_values, brightness_center2, line_color=colors[10], source=source)
    p_center.line(x_values, brightness_center3, line_color=colors[11], source=source)

    return grid

# メインループ
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv path:{csv_fpath}")

graph_height = 200
graph_width = 1000

grid = plot_fig_bokeh(csv_fpath, graph_height, graph_width)

show(grid)


In [ ]:
# グラフを作成
p_edge = figure(
    title="Upper and Lower Edge",
    sizing_mode="stretch_width",
    height=int(graph_height)
)
p_width = figure(
    title="Width",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)
p_center = figure(
    title="Brightness Center",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)

# グラフを表示する領域を作成
grid = gridplot(
    [[p_edge], [p_width], [p_center]],
    toolbar_location="above"
)

# データを追加
x_values = df_csv["ix"]
upper_edge = df_csv["trolley1_estimated_upper_edge"]
lower_edge = df_csv["trolley1_estimated_lower_edge"]
estimated_width = df_csv["trolley1_estimated_width"]
brightness_center = df_csv["trolley1_brightness_center"]

# グラフにデータを追加
# 1つ目のグラフ（Upper and Lower Edge）
p_edge.line(x_values, upper_edge, line_color="blue")
p_edge.line(x_values, lower_edge, line_color="red")

# 2つ目のグラフ（Brightness Std）
p_width.line(x_values, estimated_width, line_color="green")

# 3つ目のグラフ（Brightness Center）
p_center.line(x_values, brightness_center, line_color="orange")

show(grid)

In [ ]:
config.camera_name_to_type

In [ ]:
def set_column_names(config, trolley_dict):
    column_names = []
    for idx, trolley_id in enumerate(config.trolley_ids):
        for key in trolley_dict[img_path][trolley_id].keys():
            column_names.append(trolley_id + "_" + key)
    return column_names

column_names = set_column_names(config, trolley_dict)
pprint(f"column length: {len(column_names)}")
pprint(column_names)

In [ ]:
print(f"img_path: {img_path}")
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    pprint(f"{key} > {len(trolley_dict[img_path][config.trolley_ids[0]][key])}")

# Shelve2CSV

In [ ]:
shelve_name = "/rail.shelve"

print(f'outpath    : {outpath}')
print(f'shelve name: {shelve_name}')

In [ ]:
%%time
# 時間かかるので注意

# 格納されている名称と画像のリストを確認
with shelve.open(outpath + shelve_name, flag='r') as rail:
    print(rail['name'])
    image_list = []
    for img_path in rail[camera_num].keys():
        num_trolleyid = len(rail[camera_num][img_path].keys())
        image_list.append([img_path.split('/')[-1], num_trolleyid])
pd.DataFrame(image_list).head(5)

In [ ]:
# 格納パラメータを確認
# imagename = 'imgs/Chuo_01_Tokyo-St_up_20230201_knight/HD11/2022_0615_HD11_01_00022313.jpg'
imagename = base_images[410]

with shelve.open(outpath + shelve_name, flag='r') as rail:
    print(f'パラメータ数は{len(rail[camera_num][imagename].keys())}')
    testdict = rail[camera_num]
    for key in testdict[imagename]['trolley1'].keys():
        if len(testdict[imagename]['trolley1'][key]):
            print(f"{key}: type:{type(testdict[imagename]['trolley1'][key][0])}")
        else:
            print(f"{key}")

In [ ]:
img_idx = 410
img_path = base_images[img_idx]
print(img_path)
dir_area, camera_num = img_path.split("/")[1:3]
image_name = img_path.split('/')[-1]

columns_to_insert = [("img_idx", img_idx, 0),
                         ("dir_area", dir_area, 1),
                         ("camera_num", camera_num, 2),
                         ("image_name", image_name, 3)]
print(columns_to_insert)

In [ ]:
%%time
from tqdm import tqdm


# for test
images_path = helpers.list_imagespath_nonCache(config.image_dir)
# dir_area = "Chuo_01_Tokyo-St_up_20230201_knight"
# dir_area = "Chuo_02_Tokyo-Kanda_up_20230201_knight"
# dir_area = "Kawagoe_02_Nishioomiya-Sashiougi_down_20220626_knight"
# dir_area = "Chuo_24_Higashinakano-Nakano_down_20230201_day"
dir_area = "Takasaki_01_Omiya-Miyahara_up_20230217_day"
camera_num = "HD11"
target_dir = config.image_dir + "/" + dir_area + "/" + camera_num
outpath = config.output_dir + "/" + dir_area + "/" + camera_num
base_images = helpers.list_images(target_dir)
shelve_name = "/rail.shelve"
rail_fpath = outpath + shelve_name
print(f'outpath    : {outpath}')
print(f'shelve name: {shelve_name}')

window = 100
if window <= 2:
    window = 2
    min_periods = 1
elif window > 2:
    min_periods = int(window / 2)
print(f'window size={window}, min_periods={min_periods}')



ix_list = [int(x) for x in range(1000)]

with shelve.open(outpath + shelve_name, flag='r') as rail:
    print(f'camera_num:{camera_num}   <shelve reading...>')
    trolley_dict = rail[camera_num]

df_concat = pd.DataFrame(columns=config.columns_list)
# for idx, image_path in enumerate(trolley_dict.keys()):
for idx, image_path in tqdm(enumerate(trolley_dict.keys()), total=len(trolley_dict.keys()), desc='shelve2csv converting...'):
    # 線区名等の情報を取得
    dir_area, camera_num = image_path.split("/")[1:3]
    image_name = image_path.split('/')[-1]

    # shelveの中身をデータフレームにコピー
    for trolleyid in trolley_dict[image_path].keys():
        # print(f'trolley ID> {trolleyid}')
        df = pd.DataFrame.from_dict(trolley_dict[image_path][trolleyid], orient='index').T
        # <データ整形>データ数が1000行未満の場合に空白行を追加する
        if len(df) < 1000:
            num_rows_to_add = config.max_len - df.shape[0]
            if num_rows_to_add > 0:
                empty_df = pd.DataFrame(np.nan, index=range(num_rows_to_add), columns=df.columns)
                df = pd.concat([df, empty_df], ignore_index=True)
        # データフレームにインデックス・画像名等を挿入
        df.insert(0, 'image_idx', idx)
        df.insert(1, 'ix', ix_list)
        df['ix'] = df['ix'] + idx * 1000
        df.insert(2, 'measurement_area', dir_area)
        df.insert(3, 'camera_num', camera_num)
        df.insert(4, 'image_name', image_name)
        df.insert(5, 'trolley_id', trolleyid)
        df.insert([df.columns.get_loc(c) for c in df.columns if 'estimated_lower_edge' in c][0] + 1,
                  'estimated_width', df['estimated_lower_edge'] - df['estimated_upper_edge'])
        # df['img_idx'] = idx
        # df['ix'] = ix_list
        # df['ix'] = df['ix'] + idx * 1000
        # df['dir_area'] = dir_area
        # df['image_path'] = image_path
        # df['camera_num'] = camera_num
        # df['image_name'] = image_name
        # df['estimated_width'] = df['estimated_lower_edge'] - df['estimated_upper_edge']
        # df['trolley_id'] = trolleyid
        try:
            df_concat = pd.concat([df_concat, df], ignore_index=True)
        except:
            print("concat error -> skip")
            
print(df_concat.shape)

In [ ]:
df_concat['estimated_width_std'] = df_concat['estimated_width'].rolling(window=window, min_periods=min_periods).std()
print(df_concat.shape)
df_concat.to_csv(f'df_concat_{camera_num}.csv')
print(f'csv file name -> df_concat_{camera_num}.csv')
df_concat

In [ ]:
df_temp = df_concat.copy()
df_temp

In [ ]:
list(df_temp.columns)

In [ ]:
plt.plot(df_temp['ix'], df_temp['estimated_width'])
plt.show()

In [ ]:
window = 100

if window <= 2:
    window = 2
    min_periods = 1
elif window > 2:
    min_periods = int(window / 2)

df_temp['estimated_width_std'] = df_temp['estimated_width'].rolling(window=window, min_periods=min_periods).std()

In [ ]:
plt.plot(df_temp['ix'], df_temp['estimated_width_std'])
plt.show()

In [ ]:
ix_list = [int(x) for x in range(1000)]

img_idx = 409
image_path = base_images[img_idx]
dir_area, camera_num = image_path.split("/")[1:3]
image_name = image_path.split('/')[-1]

print(f'dir_area  :{dir_area}')
print(f'camera_num:{camera_num}')
print(f'image_path:{image_path}')

In [ ]:
%%time
for trolleyid in trolley_dict[image_path].keys():
    df = pd.DataFrame.from_dict(trolley_dict[image_path][trolleyid], orient='index').T
    df.insert(0, 'image_idx', img_idx)
    df.insert(1, 'ix', ix_list)
    df['ix'] = df['ix'] + idx * 1000
    df.insert(2, 'measurement_area', dir_area)
    df.insert(3, 'camera_num', camera_num)
    df.insert(4, 'image_name', image_name)
    df.insert(5, 'trolley_id', trolleyid)
    df.insert(8, 'estimated_width', df['estimated_lower_edge'] - df['estimated_upper_edge'])
    # df['img_idx'] = idx
    # df['ix'] = ix_list
    # df['dir_area'] = dir_area
    # df['image_path'] = image_path
    # df['camera_num'] = camera_num
    # df['image_name'] = image_name
    # df['estimated_width'] = df['estimated_lower_edge'] - df['estimated_upper_edge']
    # df['trolley_id'] = trolleyid
df

In [ ]:
df_concat.info(verbose=True)

# CSVからグラフを表示する（Shelve2CSV後の処理を想定）

In [ ]:
csv_fpath = f'df_concat_{camera_num}.csv'
print(csv_fpath)

In [ ]:
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(csv_fpath)

In [ ]:
%%time
df = pd.read_csv(csv_fpath, engine='c', dtype=config.csv_dtype)

In [ ]:
from bokeh.plotting import figure, gridplot, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, FuncTickFormatter
from bokeh.models import NumeralTickFormatter
from bokeh.palettes import d3
output_notebook()


# for test
graph_height = 200
graph_width = 600
graph_thinout = 100
ix_set_flag = False
ix_view_range = (10000, 20000)

# 初期設定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
title_text = f'Analysis data on Camera:{camera_num}'

# CSVを読み込む
# df = pd.read_csv(csv_fpath)
df = pd.read_csv(csv_fpath, engine='c', dtype=config.csv_dtype)    # 列の型を指定
print(f'csv -> DataFrame finish  df len:{len(df)}')
if not ix_set_flag:
    ix_view_range = (int(df['ix'].min()), int(df['ix'].max()))
# グラフ表示のためデータを間引く（メモリ不足…）
print(f'graph_thinout:{graph_thinout}')
df = df[::graph_thinout].copy()
print(f'thinout -> df len{len(df)}')

# グラフ描画エリアを設定
p1 = figure(width=graph_width, height=graph_height, title=title_text + ' - Estimated Edge')
p2 = figure(width=graph_width, height=graph_height, title='Width')
p3 = figure(width=graph_width, height=graph_height, title='Width Std')
p4 = figure(width=graph_width, height=graph_height, title='Brightness Std')

# グラフの要素を追加, trolley_idごとに追加
for trolleyid in df['trolley_id'].unique():
    df_temp = df.query(f'trolley_id == "{trolleyid}"').copy()
    x = df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['ix']

    p1.line(
        x, 
        df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['estimated_upper_edge'],
        legend_label=str(trolleyid)
    )
    p1.line(
        x,
        df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['estimated_lower_edge'],
        legend_label=str(trolleyid)
    )
    p2.line(
        x, 
        df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['estimated_width'], 
        legend_label=str(trolleyid)
    )
    p3.line(
        x, 
        df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['estimated_width_std'],
        legend_label=str(trolleyid)
    )
    p4.line(x,
            df_temp.query(f'{ix_view_range[0]} <= ix <= {ix_view_range[1]}')['brightness_std'],
            legend_label=str(trolleyid)
           )

# プロットを垂直に配置
layout = column(p1, p2, p3, p4)

# グラフを表示
show(layout)

In [ ]:
import matplotlib.pyplot as plt


# for test
ix_set_flag = True

# 初期設定
camera_num = 'HD21'
csv_fpath = f'df_concat_{camera_num}.csv'
graph_width = 10    # グラフ全体の幅
graph_height = 8    # グラフ全体の高さ
title_text = f'Analysis data on Camera:{camera_num}'    # タイトル共通ヘッダ

# CSVを読み込む
df = pd.read_csv(csv_fpath)
# 表示するixの範囲を指定
if ix_set_flag:
    ix = (400000, 450000)    # ixの範囲を指定する
else:
    ix = (int(df['ix'].min()), int(df['ix'].max()))

# グラフ描画エリアを設定
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(graph_width, graph_height))    # figsize (width, height)
plt.subplots_adjust(hspace=graph_height / 25)    # グラフ間の間隔を調整

fig.suptitle(title_text)
# ax1.set_xlabel('ix')
ax1.set_ylabel('estimated_edge')
# ax2.set_xlabel('ix')
ax2.set_ylabel('width')
ax3.set_xlabel('ix')
ax3.set_ylabel('brightness_std')

# グラフの要素を追加, trolley_idごとに追加
for trolleyid in df['trolley_id'].unique():
    df_temp = df.query(f'trolley_id == "{trolleyid}"').copy()
    x = df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['ix']
    ax1.plot(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_upper_edge'], label=trolleyid)
    ax1.plot(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_lower_edge'], label=trolleyid)
    ax2.plot(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_width'], label=trolleyid)
    ax3.plot(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['brightness_std'], label=trolleyid)

# 凡例を有効化する
ax1.legend()
ax2.legend()
ax3.legend()
    
plt.show()

In [ ]:
from bokeh.plotting import figure, show
# from bokeh.layouts import column
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter
from bokeh.io import output_notebook
import pandas as pd

# for test
ix_set_flag = False

# 初期設定
camera_num = 'HD21'
csv_fpath = f'df_concat_{camera_num}.csv'
graph_width = 800    # グラフ全体の幅 (Bokeh uses pixels)
graph_height = 200    # グラフ全体の高さ (Bokeh uses pixels)
title_text = f'Analysis data on Camera:{camera_num} - '    # タイトル共通ヘッダ

# CSVを読み込む
df = pd.read_csv(csv_fpath)
if ix_set_flag:
    ix = (400000, 450000)    # ixの範囲を指定する
else:
    ix = (int(df['ix'].min()), int(df['ix'].max()))

# グラフ描画エリアを設定
p1 = figure(width=graph_width, height=graph_height, title=title_text + 'Estimated Edge')
p2 = figure(width=graph_width, height=graph_height, title=title_text + 'Width', x_range = p1.x_range)
p3 = figure(width=graph_width, height=graph_height, title=title_text + 'Brightness Std', x_range = p1.x_range)

# x軸の値を千単位でカンマ区切りに設定
p1.xaxis.formatter = NumeralTickFormatter(format="0,0")
p2.xaxis.formatter = NumeralTickFormatter(format="0,0")
p3.xaxis.formatter = NumeralTickFormatter(format="0,0")

# グラフの要素を追加
for trolleyid in df['trolley_id'].unique():
    df_temp = df.query(f'trolley_id == "{trolleyid}"').copy()
    x = df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['ix']
    p1.line(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_upper_edge'], line_color='blue')
    p1.line(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_lower_edge'], line_color='red')
    p2.line(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['estimated_width'], line_color='green')
    p3.line(x, df_temp.query(f'{ix[0]} <= ix <= {ix[1]}')['brightness_std'], line_color='purple')


# プロットを垂直に配置
# layout = column(p1, p2, p3)
grid = gridplot([[p1], [p2], [p3]])

# グラフを表示
output_notebook()
# show(layout)
show(grid)
